In [1]:
import plotly.graph_objects as go
import numpy as np

def plot_tdoa_curves(sensors: np.ndarray, tdoa_vals:np.ndarray, soln: np.ndarray) -> None:
    """
    Plots TDOA hyperbolas for given sensor pairs and TDOA values.
    Args:
        sensors (np.ndarray): Array of sensor positions in meters, shape (N, 2).
        tdoa_vals (np.ndarray): Array of TDOA values corresponding to sensor pairs in meters (v*tau), shape (N-1,).
        soln (np.ndarray): Estimated source position, shape (2,).
    """

    assert sensors.shape[0] >= 2, "At least two sensors are required."
    assert sensors.shape[1] == 2, "Sensor positions must be 2D."
    assert tdoa_vals.shape[0] == sensors.shape[0] - 1, "TDOA values must match sensor pairs."
    
    sensor_pairs = [
        (sensors[0], sensors[i], tdoa_vals[i-1]) for i in range(1, len(sensors))
    ]

    fig = go.Figure()

    # scatter sensors
    for idx, s in enumerate({tuple(p) for pair in sensor_pairs for p in pair[:2]}):
        fig.add_trace(go.Scatter(
            x=[s[0]], y=[s[1]],
            mode='markers+text',
            text=[f"S{idx+1}"],
            textposition='top center',
            marker=dict(size=10),
            name=f"S{idx+1}"
        ))

    # Adding state estimate from my project images from back in the day
    fig.add_trace(go.Scatter(x=[soln[0]], y=[soln[1]] ,marker=dict(size=10,color='white'),showlegend=False))  # padding for legend

    # add hyperbolas
    x = np.linspace(-10, 10, 400)
    y = np.linspace(-10, 10, 400)
    X, Y = np.meshgrid(x, y)

    for i, (s1, s2, tdoa) in enumerate(sensor_pairs):
        r1 = np.sqrt((X - s1[0])**2 + (Y - s1[1])**2)
        r2 = np.sqrt((X - s2[0])**2 + (Y - s2[1])**2)
        F =  r2 - r1 - tdoa
        fig.add_trace(go.Contour(
            x=x, y=y, z=F,
            contours=dict(start=0, end=0, size=1, coloring='none'),
            # line=dict(width=2),
            line=dict(width=2, color='blue' if i == 0 else None),
            name=f"τ_{i+2}1"
        ))

    fig.update_layout(
        title="Multiple TDOA Hyperbolas",
        xaxis=dict(scaleanchor='y', scaleratio=1)
    )

    fig.show()

In [4]:
sensors = np.array([
    [2, 2],
    [8, -8],
    [-8, 0]
])

# Test case 1 from the cpp re-implementation
tdoa_vals = np.array([1.2, 3.2])
soln = np.array([0.7988, -4.639])

# Test case 2 from the cpp re-implementation
# tdoa_vals = np.array([1.2, -3.2])
# soln = np.array([-4.4452, -7.2429])

# Test case 3 from the cpp re-implementation
# tdoa_vals = np.array([6.12684, -1.01795])
# soln = np.array([-3, -2])

plot_tdoa_curves(sensors, np.array(tdoa_vals), soln)